https://www.youtube.com/watch?v=lVXJgn3fDkg

In [1]:
import qiskit
from qiskit import QuantumCircuit, transpile, execute
from qiskit.extensions import UnitaryGate
from qiskit.providers.aer import QasmSimulator
from qiskit.tools.visualization import plot_bloch_multivector, plot_histogram
from qiskit import IBMQ
from qiskit.tools.monitor import job_monitor

import numpy as np
import matplotlib.pyplot as plt
import warnings
import math
warnings.filterwarnings('ignore')

In [2]:
from qiskit.tools.visualization import plot_bloch_multivector, plot_histogram

$$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$$
$$\newcommand{\bra}[1]{\left\langle{#1}\right|}$$
$$\newcommand{\braket}[2]{\left\langle{#1}\middle|{#2}\right\rangle}$$

In [3]:
# Display measurement results
def show_barchart(counts):
    y_pos=np.arange(2)
    objects=['a','b']
    plt.xticks(y_pos, objects)
    plt.bar(y_pos, counts)
def show_statevector(statevector):
    print(f'statevector {np.around(statevector[0],3)} {np.around(statevector[1],3)}')
def get_probabilities(statevector):
    n_uu = np.abs(statevector[0])**2
    n_du = np.abs(statevector[1])**2
    n_ud = np.abs(statevector[2])**2
    n_dd = np.abs(statevector[3])**2
    return n_uu, n_du, n_ud, n_dd


## Verify that the matrix for the gate is unitary

In [4]:
def unitary(U):
    assert(np.around(np.matmul(np.conj(np.transpose(U)),U),5)==np.eye(2)).all()

## Gate for non-vertical measurement basis

The rows of the unitary matrix for the measurement are the complex conjugates of the states of the measurement basis, which is in the yz plane

In [5]:
def measurement_gate(theta, show):
    U=np.array([[math.cos(theta/2),math.sin(theta/2)],\
                [math.sin(theta/2),-math.cos(theta/2)]])
    if show:
        print('unitary matrix for projection')
        print(U[0,0],U[0,1])
        print(U[1,0],U[1,1])
    unitary(U)
    return U
    

## Project state vector onto new measurement basis

In [6]:
def project(bit,circuit, theta, show):
    U = measurement_gate(theta, show)
    circuit.unitary(U,[bit])
    return circuit

## Measure with polarization not vertical

In [7]:
def measure(bit, circuit, angle, show=False):
    circuit = project(bit, circuit, angle, show)
    circuit.measure(bit,bit)
    return circuit

In [8]:
def get_encoding(keybit, basis):
    return 2*keybit+basis

In [9]:
def prepare_state(keybit, basis):
    circuit = qiskit.QuantumCircuit(1,1)
    encoding = get_encoding(keybit, basis)
    if encoding == 0:
#             zero degrees
        pass
    elif encoding == 1:
#             90 degrees
        circuit.h(0)
    elif encoding == 2:
#             180 degrees
        circuit.x(0)
    else:
#             270 degrees
        circuit.x(0)
        circuit.h(0)
    return circuit

In [10]:
show_vector = False
simulate = True
if show_vector:
    backend = qiskit.Aer.get_backend('statevector_simulator')
else:
    backend = QasmSimulator()
circuit=qiskit.QuantumCircuit(2,2)
thetas=math.radians(0), math.radians(90), math.radians(180), math.radians(270)

## Alice draws a random sequence of key bits

## Alice draws a random sequence of measurement basis vectors

## Bob draws a random sequence of measurement basis vactors

In [11]:
keysize=80
np.random.seed(0)
alice_key=np.random.randint(2,size=keysize)
bob_key = np.ndarray(keysize, dtype=int)
alice_bases = np.random.randint(2, size=keysize)
bob_bases = np.random.randint(2, size=keysize)

## Alice encodes her choices of key bits and measurement basis vectors into qubits

## Alice transmits her qubits to Bob

## Bob measures Alice's qubits in his chosen measurement bases

## Alice and Bob each publish their chosen measurement bases

In [12]:
for i, keybit in enumerate(alice_key):
    alice_basis = alice_bases[i]
    bob_basis = bob_bases[i]

#     Alice prepares a qubit
    circuit = prepare_state(keybit, alice_basis)
    result = qiskit.execute(circuit, backend=backend).result()
    
    if show_vector:
        print()
        statevector = result.get_statevector()
        show_statevector(statevector)
    else:
        
#         Bob measures Alice's qubit in his chosen measurement basis
        if bob_basis == 0:
            circuit = measure(0, circuit, 0)
        else:
            circuit = measure(0, circuit, math.radians(90))
        compiled_circuit = transpile(circuit)
        job = backend.run(compiled_circuit, shots=1)
        result = job.result()
        counts = result.get_counts(circuit)
        dict_keys = list(counts.keys())
        if dict_keys[0]=='0':
            bob_keybit = 0
        else:
            bob_keybit = 1
        bob_key[i]=bob_keybit

## Where Alice and Bob chose the same measurement basis, Alice's key bit matches Bob's key bit

In [13]:
for i in range(keysize):
    alice_basis = alice_bases[i]
    bob_basis = bob_bases[i]
    if alice_basis == bob_basis:
        assert(alice_key[i]==bob_key[i])
        